In [1]:
from datetime import datetime
print("COMENZADO A LAS :")
datetime.now().time()

COMENZADO A LAS :


datetime.time(14, 7, 24, 521212)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

files = os.listdir('data_preprocessed_python\\')

list_of_dfs = []
for file in files:
    with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
        
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        allData = allData.merge(aux, left_index=True, right_index=True)
        
        list_of_dfs.append(allData)
big_df = pd.concat(list_of_dfs, ignore_index=True)#ignore_index to reset index of big_df


In [2]:
data = big_df.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence']

data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

data


,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz,valence
0,0.948232,10.260175,1.013050,-7.658428,-1.811108,11.011411,3.026008,-2.380048,3.978952,-9.657708,...,1.134884,1.435263,1.388486,4.239575,4.888393,0.596471,0.589618,-2.276449,-0.109300,happy
1,1.653335,12.795443,-1.067832,-3.267558,-4.783876,7.402976,2.676232,-3.614201,-1.434440,-0.906298,...,1.824904,-1.665933,-1.577337,4.557239,6.007259,-1.881391,-4.831903,-1.739787,-6.518661,happy
2,3.013726,10.426192,3.908249,0.701542,-0.522649,1.120469,2.046996,-4.286566,-6.174767,9.408599,...,-4.090714,1.008985,-6.161669,0.636801,2.921478,-4.484906,-8.186728,0.555901,-11.727187,happy
3,1.495061,8.229207,6.094405,2.959722,1.299854,-0.832024,2.192056,-5.170547,-6.571542,14.101073,...,-2.502785,0.562838,-4.305524,-2.965047,-0.679860,-4.483488,-7.905437,-1.168129,-9.051847,happy
4,-1.264836,3.751782,4.145906,3.459897,-0.916779,-0.784404,-4.694002,-5.332026,-7.793136,16.284187,...,6.397443,-3.709545,-0.162134,3.248115,-1.103246,-2.547386,-4.419073,-1.319219,-4.072682,happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10321915,-32.686626,-118.432236,107.674687,71.185171,-81.400159,-31.566511,128.195564,-93.473053,151.210251,-68.675085,...,-33.819192,229.757144,28.856880,-85.715499,-205.343696,-77.855162,3.823642,39.977058,-161.983832,happy
10321916,-0.499185,-151.650924,62.237271,110.122599,-118.743840,69.402053,75.226912,-70.535596,183.491442,-53.675112,...,-16.694224,233.225887,25.325637,-39.621834,-223.718662,-38.542735,38.761077,27.164582,-181.327546,happy
10321917,-64.780317,-113.557245,87.924724,107.091355,-64.368940,166.745623,103.226860,-97.848045,177.491453,-102.675022,...,1.618242,240.132125,14.575657,-47.309320,-222.187415,-45.323972,21.011110,74.945744,-162.327581,happy
10321918,-5.030427,-136.619702,80.080988,98.872620,-113.806349,203.276805,96.226873,-93.723053,184.335190,-56.362607,...,18.180712,204.538440,-11.705545,46.628006,-220.749918,-69.448928,20.104862,19.914595,-171.140065,happy


In [3]:
from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

# Random Forest

In [4]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, yTrain)

<ipython-input-4-353746cc4ea1>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  3.2min
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:  7.9min finished


RandomForestClassifier(max_depth=2, n_jobs=5, random_state=0, verbose=1)

In [5]:
predict = clf.predict(xTest)

from sklearn.metrics import accuracy_score

accuracy_score(yTest, predict)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    1.6s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    4.2s finished


0.650055900452629

# SVM

In [ ]:
printf("SVM EMPEZADO A LAS :")
datetime.now().time()

In [6]:
from sklearn import svm
from sklearn.svm import LinearSVC

svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)
accuracy_score(yTest, svm_predict)

In [ ]:
printf("SVM terminado a las A LAS :")
datetime.now().time()

# kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 7

knn = KNeighborsClassifier(n_neighbors)
knn.fit(xTrain, np.ravel(yTrain))
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(xTrain, yTrain)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(xTest, yTest)))